In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [4]:
data = pd.read_csv('~/pristavki.csv', header=None, names=['text'])

In [6]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [7]:
Game = fact(
    'Game',
    ['title', 'subtitle', 'series_number']
)

In [33]:
CoD_subtitles = {
    'MW': 'Modern Warfare',
    'Black Ops': 'Black Ops',
    'Infinite Warfare': 'Infinite Warfare',
    'Advanced Warfare': 'Advanced Warfare',
    'Modern Warfare': 'Modern Warfare',
    'World at War': 'World at War',
    'Ghosts': 'Ghosts'
}

CoD_series_numbers = {
    'III': '3',
    '1': '1',
    '2': '2',
    '3': '3',
}

CoD = rule(
    morph_pipeline(['Call of Duty']).interpretation(Game.title.const('Call of Duty:')),
    morph_pipeline(['Black Ops', 'Infinite Warfare', 'Advanced Warfare', 'Modern Warfare', 'MW', 'World at War', 'Ghosts']).interpretation(Game.subtitle.custom(CoD_subtitles.get)).optional(),
    morph_pipeline(['1', '2', '3', 'III']).interpretation(Game.series_number.custom(CoD_series_numbers.get)).optional(),
)

In [34]:
Fifa = rule(
    morph_pipeline(['Fifa', 'фифа']).interpretation(Game.title.const('FIFA')),
    morph_pipeline(['16', '11', '15', '2004']).interpretation(Game.series_number).optional(),
)

In [35]:
GAME = or_(CoD,Fifa).interpretation(Game)

parser = Parser(GAME)

In [36]:
matches = []

for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [38]:
print(len(matches))
# for m in matches:
#     print(m.title, m.subtitle, m.series_number)

707


In [ ]:
Метод позволяет находить нужные сущности, что заданы правилами, 
при этом, что интересно, он позволяет находить их даже в случае, если в середине встречаются лишние символы:
так, прочитав "Call of Duty: Black Ops", он находит название игры, хотя там есть и лишнее двоеточие
Также стоит отметить, что отсутствие пробелов между частями сущности не мешает находить сущность и ее парсить:
так, "фифа15" преобразуется в "FIFA None 15"